Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru.
Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы).
Получившийся список должен содержать в себе минимум:

*Наименование вакансии
*Предлагаемую зарплату (отдельно мин. отдельно макс. и отдельно валюту)
*Ссылку на саму вакансию
*Сайт откуда собрана вакансии

In [5]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [6]:
def hh(vacancy):
    vacancy_date = []
    params = {
        'text': vacancy,
        'search_field': 'name',
        'items_on_page': '100',
        'page': ''
    }

    link = 'https://hh.ru/search/vacancy'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 YaBrowser/20.4.1.225 Yowser/2.5 Safari/537.36'
    }
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text, 'html.parser')
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())

    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)

    return vacancy_date

In [7]:
def hh_item(item):
    vacancy_date = {}

    #name
    vacancy_name = item.find('span', {'class': 'g-user-content'}).getText()
    vacancy_date['vacancy_name'] = vacancy_name

    #salary
    vacancy_salary = item.find('div', {'class': 'vacancy-serp-item__sidebar'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText()

        salary = re.split(r'\s|-', salary)

        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[1])

        salary_currency = salary[2]
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency

    #link
    vacancy_link = item.find('span', {'class': 'g-user-content'}).find('a')['href']
    vacancy_date['link'] = vacancy_link

    #website
    vacancy_website = 'hh.ru'
    vacancy_date['website'] = vacancy_website

    return vacancy_date

In [8]:
def superjob(vacancy):
    vacancy_date = []
    
    params = {
        'keywords': vacancy,
        'profession_only': '1',
        'geo[c][0]': '15',
        'geo[c][1]': '1',
        'geo[c][2]': '9',
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 YaBrowser/20.4.1.225 Yowser/2.5 Safari/537.36'
    }
    
    link = 'https://www.superjob.ru/vacancy/search/'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
    
        page_block = parsed_html.find('a', {'class': 'f-test-button-1'})
    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())
    
    for page in range(0, last_page + 1):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
                
    return vacancy_date

In [9]:
def superjob_item(item):

    vacancy_date = {}
    
    # vacancy_name
    vacancy_name = item.find_all('a')
    if len(vacancy_name) > 1:
        vacancy_name = vacancy_name[-2].getText()
    else:
        vacancy_name = vacancy_name[0].getText()
    vacancy_date['vacancy_name'] = vacancy_name
    
    #salary
    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}).findChildren()
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary_currency = salary[-1].getText()
        is_check_salary = item.find('span', {'class': 'f-test-text-company-item-salary'}).getText()
        
        if is_check_salary == 'до' or len(salary) == 2:
            salary_min = None
            salary_max = int(salary[0]).getText()
        
        elif is_check_salary == 'от':
            salary_min = int(salary[0]).getText()
            salary_max = None
        
        else:
            salary_min = int(salary[0]).getText()
            salary_max = int(salary[2]).getText()          
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    
    # link
    vacancy_link = item.find_all('a')
    
    if len(vacancy_link) > 1:
        vacancy_link = vacancy_link[-2]['href']
    else:
        vacancy_link = vacancy_link[0]['href']
    
    vacancy_date['vacancy_link'] = f'https://www.superjob.ru{vacancy_link}'
    
    # website
    vacancy_date['website'] = 'www.superjob.ru'
    
    return vacancy_date

In [10]:
def parser_vacancy(vacancy):
    vacancy_date = []
    vacancy_date.append(hh(vacancy))
    vacancy_date.append(superjob(vacancy))
    
    df = pd.DataFrame(vacancy_date)

In [11]:
vacancy = 'Python'
df = parser_vacancy(vacancy)